In [ ]:
%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image, ImageEnhance, ImageFilter
import matplotlib.pyplot as plt
from io import BytesIO
import pandas as pd
import time
import numpy as np

# ---------------------------- DEVICE ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------------------------- UTILS ----------------------------
@st.cache_data
def pil_to_tensor(img: Image.Image, max_size=512, shape=None):
    img = img.convert('RGB')
    if shape:
        img = img.resize(shape, Image.LANCZOS)
    else:
        size = min(max(img.size), max_size)
        img.thumbnail((size, size), Image.LANCZOS)
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
    ])
    return transform(img).unsqueeze(0).to(device)

def load_uploaded_image(uploaded_file, max_size=512, shape=None):
    """Fix lỗi UploadedFile không có .convert()"""
    img = Image.open(uploaded_file).convert("RGB")
    return pil_to_tensor(img, max_size=max_size, shape=shape)

def im_convert(tensor, max_display_size=400):
    image = tensor.clone().detach().cpu().squeeze(0)
    mean = torch.tensor([0.485,0.456,0.406]).view(3,1,1)
    std = torch.tensor([0.229,0.224,0.225]).view(3,1,1)
    image = image * std + mean
    image = torch.clamp(image,0,1)
    img = transforms.ToPILImage()(image)
    img.thumbnail((max_display_size, max_display_size), Image.LANCZOS)
    return img

def pil_from_tensor(tensor):
    return im_convert(tensor, max_display_size=1000)

def gram_matrix(tensor):
    b, c, h, w = tensor.size()
    tensor = tensor.view(b, c, h*w)
    return torch.bmm(tensor, tensor.transpose(1,2)) / (c*h*w)

# ---------------------------- POST-PROCESSING ----------------------------
def postprocess_pil(img: Image.Image, sharpen=0.0, tone=1.0, smooth=0.0, hdr=0.0):
    if smooth > 0:
        img = img.filter(ImageFilter.SMOOTH_MORE)
    if sharpen > 0:
        enhancer = ImageEnhance.Sharpness(img)
        img = enhancer.enhance(1.0 + sharpen)
    if tone != 1.0:
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(tone)
    if hdr > 0:
        enhancer = ImageEnhance.Contrast(img)
        hdr_img = enhancer.enhance(1.5 + hdr)
        img = Image.blend(img, hdr_img, min(0.6, hdr))
    return img

# ---------------------------- FEATURE EXTRACTOR ----------------------------
class VGGFeatures(nn.Module):
    def __init__(self, content_layers, style_layers):
        super().__init__()
        vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).features
        self.vgg = nn.Sequential(*list(vgg.children())[:29]).to(device).eval()
        for p in self.vgg.parameters():
            p.requires_grad = False
        self.content_layers = content_layers
        self.style_layers = style_layers

    def forward(self, x):
        content_features, style_features = {}, {}
        for name, layer in self.vgg._modules.items():
            x = layer(x)
            if name in self.content_layers:
                content_features[name] = x
            if name in self.style_layers:
                style_features[name] = x
        return content_features, style_features

# ---------------------------- STREAMLIT APP ----------------------------
st.set_page_config(page_title="🎨 Neural Style Transfer", layout="wide")
st.title("🎨 Neural Style Transfer - Chuyển Phong Cách Nghệ Thuật")

# ---------------------------- Session state ----------------------------
if 'loss_table' not in st.session_state:
    st.session_state.loss_table = pd.DataFrame(columns=["Step","Total Loss","Content Loss","Style Loss","Optimizer"])
# per-optimizer summary tables (mỗi hàng = mốc 100 step trung bình / hoặc mốc cuối)
if 'loss_table_lbfgs' not in st.session_state:
    st.session_state.loss_table_lbfgs = pd.DataFrame(columns=["Step","Avg Total","Avg Content","Avg Style"])
if 'loss_table_adam' not in st.session_state:
    st.session_state.loss_table_adam = pd.DataFrame(columns=["Step","Avg Total","Avg Content","Avg Style"])
if "target_img" not in st.session_state: st.session_state.target_img = None
if "adam_target_img" not in st.session_state: st.session_state.adam_target_img = None
if "timings" not in st.session_state: st.session_state.timings = {}

# ---------------------------- Tabs ----------------------------
tab1, tab2, tab3, tab4 = st.tabs(["Upload & Settings", "Training Progress", "Result & Download", "So sánh Optimizers"])

# ---------------------------- Sidebar: Upload & Settings ----------------------------
with st.sidebar:
    st.header("📂 Upload Ảnh")
    content_file = st.file_uploader("Ảnh Nội Dung", type=["jpg","png"])
    style_files = st.file_uploader("Ảnh Phong Cách (nhiều)", type=["jpg","png"], accept_multiple_files=True)

    st.header("⚙️ Tham số")
    # thêm 500
    steps_choice = st.selectbox("Steps", [100,200,300,400,500], index=1)
    downsample = st.slider("Kích thước tối đa", 256, 1024, 512, step=64)
    alpha = st.slider("α (Content)", 0.1, 10.0, 1.0, 0.1)
    beta = st.slider("β (Style)", 1e3, 1e5, 1e4, step=1e3, format="%.0f")

    st.header("Optimizers")
    run_lbfgs = st.checkbox("Dùng L-BFGS", True)
    run_adam = st.checkbox("Dùng Adam", True)
    adam_lr = st.slider("Adam LR", 1e-3, 1e-1, 1e-2, format="%.4f")

    st.header("Hậu xử lý")
    sharpen = st.slider("Sharpen", 0.0, 2.0, 0.0)
    tone = st.slider("Tone", 0.5, 2.0, 1.0)
    smooth = st.slider("Smooth", 0.0, 2.0, 0.0)
    hdr = st.slider("HDR", 0.0, 1.0, 0.0)

    start_button = st.button("🚀 Start")
    reset_button = st.button("♻ Reset")

    # Nút xem giải thích tham số — dùng expander để không làm rối giao diện
    with st.expander("🔎 Giải thích các tham số (bấm để mở)"):
        st.markdown("""
        - **α (Content)**: hệ số cho content loss. **Tăng** nếu muốn giữ nhiều kết cấu/nội dung gốc hơn; **giảm** nếu muốn ảnh hưởng của phong cách mạnh hơn.
        - **β (Style)**: hệ số cho style loss. **Tăng** để áp phong cách rõ rệt hơn; **giảm** nếu phong cách quá áp đảo.
        - **Steps**: số vòng tối ưu. Nhiều bước → ảnh mượt và ổn định hơn nhưng tốn thời gian.
        - **Adam LR**: learning rate cho Adam. Nếu loss không giảm → thử giảm LR; nếu quá chậm → có thể tăng nhẹ (cẩn trọng).
        - **Sharpen / Tone / Smooth / HDR**: hậu xử lý ảnh cuối cùng — điều chỉnh trực quan sau khi train.
        """)
        st.markdown("**Gợi ý**: nếu muốn nhiều chi tiết phong cách nhỏ, tăng beta và/hoặc tăng steps; nếu muốn bảo tồn bố cục, tăng alpha.")

# ---------------------------- RESET ----------------------------
if reset_button:
    st.session_state.loss_table = pd.DataFrame(columns=["Step","Total Loss","Content Loss","Style Loss","Optimizer"])
    st.session_state.loss_table_lbfgs = pd.DataFrame(columns=["Step","Avg Total","Avg Content","Avg Style"])
    st.session_state.loss_table_adam = pd.DataFrame(columns=["Step","Avg Total","Avg Content","Avg Style"])
    st.session_state.target_img = None
    st.session_state.adam_target_img = None
    st.session_state.timings = {}
    st.experimental_rerun()

# ---------------------------- Tab 1 ----------------------------
with tab1:
    st.subheader("Upload & Preview")
    cols = st.columns(2)
    # Hiển thị content và blended style cùng hàng
    if content_file is not None:
        try:
            content_preview = Image.open(content_file).convert("RGB")
            cols[0].image(content_preview, caption="Content", use_column_width=True)
        except Exception:
            cols[0].write("Không thể hiển thị ảnh nội dung.")
    else:
        cols[0].info("Chưa upload ảnh nội dung")

    if style_files and len(style_files) > 0:
        try:
            # Blend thumbnails for preview
            thumbs = [Image.open(f).convert("RGB").resize((256,256), Image.LANCZOS) for f in style_files]
            # tạo 1 hàng ảnh phong cách nhỏ
            row_w = sum(t.width for t in thumbs)
            blend_canvas = Image.new("RGB", (256*len(thumbs), 256))
            for i, t in enumerate(thumbs):
                blend_canvas.paste(t, (i*256,0))
            cols[1].image(blend_canvas, caption="Blended Style (preview)", use_column_width=True)
        except Exception:
            cols[1].write("Không thể hiển thị ảnh phong cách.")
    else:
        cols[1].info("Chưa upload ảnh phong cách")

# ---------------------------- START NST ----------------------------
if start_button:
    if content_file is None or len(style_files) == 0:
        st.error("Bạn phải chọn ảnh nội dung và ít nhất 1 ảnh phong cách!")
        st.stop()

    content = load_uploaded_image(content_file, max_size=downsample)

    # FIX LỖI TẠI ĐÂY — convert UploadedFile → PIL.Image đúng chuẩn
    style_tensors = [
        load_uploaded_image(f, max_size=downsample, shape=tuple(content.shape[-2:][::-1]))
        for f in style_files
    ]
    style = torch.mean(torch.stack(style_tensors), dim=0)

    # show small previews again inside tab1 (kept UI behavior)
    with tab1:
        c1, c2 = st.columns(2)
        c1.image(im_convert(content), caption="Content")
        c2.image(im_convert(style), caption="Blended Style")

    content_layers = ["21"]
    style_layers = ["0","5","10","19","28"]
    extractor = VGGFeatures(content_layers, style_layers)

    content_feat, _ = extractor(content)
    style_feat = extractor(style)[1]
    style_grams = {l: gram_matrix(style_feat[l]) for l in style_layers}

    # Prepare progress UI placeholders in tab2
    with tab2:
        st.subheader("Đang chạy tối ưu…")
        # Two columns: LBFGS / Adam progress + tables
        pcols = st.columns(2)
        lbfgs_status = pcols[0].empty()
        lbfgs_progress = pcols[0].progress(0)
        adam_status = pcols[1].empty()
        adam_progress = pcols[1].progress(0)

        # Two dataframes to display (per-optimizer summary mốc 100)
        tcols = st.columns(2)
        lbfgs_table_placeholder = tcols[0].empty()
        adam_table_placeholder = tcols[1].empty()

    # ---------------------------- OPTIMIZATION ----------------------------
    def run_lbfgs(target, steps, progress_bar=None, status_placeholder=None, table_placeholder=None):
        optimizer = torch.optim.LBFGS([target], max_iter=1)
        c_losses, s_losses, totals = [], [], []
        start = time.time()
        interval_size = 100
        last_interval_index = 0
        for step in range(steps):
            def closure():
                optimizer.zero_grad()
                t_content, t_style = extractor(target)
                c_loss = torch.mean((t_content["21"] - content_feat["21"])**2)
                s_loss = sum(torch.mean((gram_matrix(t_style[l]) - style_grams[l])**2) for l in style_layers)
                loss = alpha*c_loss + beta*s_loss
                loss.backward()
                c_losses.append(c_loss.item()); s_losses.append(s_loss.item()); totals.append(loss.item())
                return loss
            optimizer.step(closure)

            # update progress UI
            if progress_bar is not None:
                pct = int(((step+1)/steps)*100)
                progress_bar.progress(pct)
            if status_placeholder is not None:
                status_placeholder.info(f"L-BFGS: step {step+1}/{steps}  — loss {totals[-1]:.4f}")

            # every interval_size steps or final step -> add summary row
            if ((step+1) % interval_size == 0) or (step+1 == steps):
                seg_tot = totals[last_interval_index:step+1]
                seg_c = c_losses[last_interval_index:step+1]
                seg_s = s_losses[last_interval_index:step+1]
                if len(seg_tot) > 0:
                    avg_tot = float(np.mean(seg_tot))
                    avg_c = float(np.mean(seg_c))
                    avg_s = float(np.mean(seg_s))
                    mstep = step+1
                    st.session_state.loss_table_lbfgs.loc[len(st.session_state.loss_table_lbfgs)] = [mstep, avg_tot, avg_c, avg_s]
                    # update table display
                    if table_placeholder is not None:
                        table_placeholder.dataframe(st.session_state.loss_table_lbfgs, use_container_width=True)
                last_interval_index = step+1

        return target.detach(), c_losses, s_losses, totals, time.time() - start

    def run_adam(target, steps, lr, progress_bar=None, status_placeholder=None, table_placeholder=None):
        optimizer = torch.optim.Adam([target], lr=lr)
        c_losses, s_losses, totals = [], [], []
        start = time.time()
        interval_size = 100
        last_interval_index = 0
        for step in range(steps):
            optimizer.zero_grad()
            t_content, t_style = extractor(target)
            c_loss = torch.mean((t_content["21"] - content_feat["21"])**2)
            s_loss = sum(torch.mean((gram_matrix(t_style[l]) - style_grams[l])**2) for l in style_layers)
            loss = alpha*c_loss + beta*s_loss
            loss.backward()
            optimizer.step()
            c_losses.append(c_loss.item()); s_losses.append(s_loss.item()); totals.append(loss.item())

            # update progress UI
            if progress_bar is not None:
                pct = int(((step+1)/steps)*100)
                progress_bar.progress(pct)
            if status_placeholder is not None:
                status_placeholder.info(f"Adam: step {step+1}/{steps}  — loss {totals[-1]:.4f}")

            # every interval_size steps or final step -> add summary row
            if ((step+1) % interval_size == 0) or (step+1 == steps):
                seg_tot = totals[last_interval_index:step+1]
                seg_c = c_losses[last_interval_index:step+1]
                seg_s = s_losses[last_interval_index:step+1]
                if len(seg_tot) > 0:
                    avg_tot = float(np.mean(seg_tot))
                    avg_c = float(np.mean(seg_c))
                    avg_s = float(np.mean(seg_s))
                    mstep = step+1
                    st.session_state.loss_table_adam.loc[len(st.session_state.loss_table_adam)] = [mstep, avg_tot, avg_c, avg_s]
                    if table_placeholder is not None:
                        table_placeholder.dataframe(st.session_state.loss_table_adam, use_container_width=True)
                last_interval_index = step+1

        return target.detach(), c_losses, s_losses, totals, time.time() - start

    # RUN BOTH (vẫn giữ logic cũ cho training)
    with tab2:
        # reset global combined table for plotting later (optional)
        st.session_state.loss_table = pd.DataFrame(columns=["Step","Total Loss","Content Loss","Style Loss","Optimizer"])

        if run_lbfgs:
            lbfgs_status.info("🔵 L-BFGS started")
            t_lbfgs, c_l, s_l, tot_l, elapsed = run_lbfgs(content.clone().requires_grad_(True), steps_choice,
                                                         progress_bar=lbfgs_progress,
                                                         status_placeholder=lbfgs_status,
                                                         table_placeholder=lbfgs_table_placeholder)
            st.session_state.target_img = t_lbfgs
            st.session_state.timings["LBFGS"] = elapsed
            # append per-step to combined table (kept for plotting / record)
            for i in range(len(tot_l)):
                st.session_state.loss_table.loc[len(st.session_state.loss_table)] = [i+1,tot_l[i],c_l[i],s_l[i],"LBFGS"]
            lbfgs_status.success("🔵 L-BFGS done")

        if run_adam:
            adam_status.info("🟠 Adam started")
            t_adam, c_a, s_a, tot_a, elapsed = run_adam(content.clone().requires_grad_(True), steps_choice, adam_lr,
                                                       progress_bar=adam_progress,
                                                       status_placeholder=adam_status,
                                                       table_placeholder=adam_table_placeholder)
            st.session_state.adam_target_img = t_adam
            st.session_state.timings["Adam"] = elapsed
            for i in range(len(tot_a)):
                st.session_state.loss_table.loc[len(st.session_state.loss_table)] = [i+1,tot_a[i],c_a[i],s_a[i],"Adam"]
            adam_status.success("🟠 Adam done")

        st.success("🎉 Hoàn tất!")

# ---------------------------- Tab 3: RESULT & DOWNLOAD ----------------------------
with tab3:
    st.subheader("Ảnh kết quả")
    cols = st.columns(2)
    if st.session_state.target_img is not None:
        img = postprocess_pil(pil_from_tensor(st.session_state.target_img), sharpen, tone, smooth, hdr)
        cols[0].image(img, caption="L-BFGS")
        # download button
        buf = BytesIO()
        img.save(buf, format="PNG")
        buf.seek(0)
        cols[0].download_button("Tải L-BFGS", data=buf, file_name="result_lbfgs.png", mime="image/png")
    else:
        cols[0].info("Chưa có kết quả L-BFGS")

    if st.session_state.adam_target_img is not None:
        img2 = postprocess_pil(pil_from_tensor(st.session_state.adam_target_img), sharpen, tone, smooth, hdr)
        cols[1].image(img2, caption="Adam")
        buf2 = BytesIO()
        img2.save(buf2, format="PNG")
        buf2.seek(0)
        cols[1].download_button("Tải Adam", data=buf2, file_name="result_adam.png", mime="image/png")
    else:
        cols[1].info("Chưa có kết quả Adam")

    if not st.session_state.loss_table.empty:
        st.subheader("Biểu đồ Loss (Total)")
        fig, ax = plt.subplots()
        for opt in st.session_state.loss_table["Optimizer"].unique():
            df = st.session_state.loss_table[st.session_state.loss_table["Optimizer"]==opt]
            ax.plot(df["Total Loss"].values, label=opt)
        ax.legend()
        st.pyplot(fig)

    # show the per-100-step summary tables (side-by-side)
    st.subheader("Bảng tóm tắt mỗi 100 step (hoặc mốc cuối)")
    scols = st.columns(2)
    scols[0].markdown("**L-BFGS**")
    scols[1].markdown("**Adam**")
    scols[0].dataframe(st.session_state.loss_table_lbfgs if not st.session_state.loss_table_lbfgs.empty else pd.DataFrame({"Info":["Chưa có dữ liệu"]}))
    scols[1].dataframe(st.session_state.loss_table_adam if not st.session_state.loss_table_adam.empty else pd.DataFrame({"Info":["Chưa có dữ liệu"]}))

# ---------------------------- Tab 4: COMPARE ----------------------------
with tab4:
    st.header("So sánh L-BFGS vs Adam")

    if "LBFGS" in st.session_state.timings:
        st.write(f"⏱️ L-BFGS: {st.session_state.timings['LBFGS']:.2f}s")
    if "Adam" in st.session_state.timings:
        st.write(f"⏱️ Adam: {st.session_state.timings['Adam']:.2f}s")

    if not st.session_state.loss_table.empty:
        st.subheader("Biểu đồ riêng Content/Style/Total")
        fig, axs = plt.subplots(3,1, figsize=(6,10))
        for idx, loss_name in enumerate(["Content Loss","Style Loss","Total Loss"]):
            for opt in st.session_state.loss_table["Optimizer"].unique():
                df = st.session_state.loss_table[st.session_state.loss_table["Optimizer"]==opt]
                axs[idx].plot(df[loss_name].values, label=opt)
            axs[idx].set_ylabel(loss_name)
            axs[idx].legend()
        st.pyplot(fig)
    # nút để xem công thức/khái niệm
    if st.button("📘 Xem khái niệm Adam vs L-BFGS"):
        with st.expander("Adam & L-BFGS là gì? (Khái niệm đơn giản)"):
            st.markdown("""
            ## 🔵 L-BFGS (Limited-memory BFGS)
            - Là thuật toán tối ưu **dựa trên quasi-Newton**, dùng xấp xỉ đạo hàm bậc hai (curvature).
            - Ra quyết định bước đi tối ưu **dựa trên hình dạng bề mặt loss**, không chỉ dựa trên gradient đơn thuần.
            - Thường **hội tụ nhanh** và **ổn định** trong các bài toán tối ưu ảnh (image optimization).
            - Rất phù hợp với **Neural Style Transfer**, theo bài báo gốc của Gatys.

            ---

            ## 🟠 Adam (Adaptive Moment Estimation)
            - Là thuật toán tối ưu **gradient descent hiện đại**, có cơ chế:
                - momentum (ổn định hướng đi)
                - adaptive learning rate (mỗi tham số có tốc độ riêng)
            - Rất mạnh trong **huấn luyện mô hình lớn** (deep learning), nhưng trong NST:
                - kết quả phụ thuộc nhiều vào learning rate
                - đôi khi kém sắc nét hơn L-BFGS
            - Ưu điểm: dễ điều chỉnh, linh hoạt, tốc độ mỗi step nhanh.

            """)

        with st.expander("So sánh nhanh Adam vs L-BFGS"):
            st.markdown("""
            ## 📊 Bảng so sánh

            | Tiêu chí | L-BFGS | Adam |
            |---------|--------|------|
            | Tốc độ hội tụ | ⭐ **Rất nhanh** | Trung bình |
            | Độ sắc nét (NST) | ⭐ **Thường cao hơn** | Có thể mềm, mượt hơn |
            | Tính ổn định | ⭐ Cao | Trung bình |
            | Dễ tinh chỉnh | Ít (khó điều chỉnh nhưng mặc định tốt) | ⭐ Dễ điều chỉnh (LR) |
            | Phù hợp cho NST | ⭐⭐ **Rất phù hợp** | Hợp nhưng không tối ưu |
            | Phù hợp DL nói chung | Không | ⭐⭐ **Rất phù hợp** |
            | Phụ thuộc LR | Không | ⭐ Có |

            """)

        with st.expander("Kết luận — thuật toán nào tối ưu hơn cho NST?"):
            st.markdown("""
            ## 🏆 Kết luận

            - **L-BFGS thường cho kết quả tốt hơn trong Neural Style Transfer**:
                - ảnh sắc nét hơn
                - hội tụ nhanh hơn
                - ít phụ thuộc hyperparameter
                - được dùng trong bài báo gốc

            - **Adam**:
                - tốt trong huấn luyện mô hình lớn
                - nhưng trong NST, cần điều chỉnh LR kỹ nếu không ảnh có thể bị mờ hoặc nhiễu

            👉 **Nếu mục tiêu của bạn là chất lượng ảnh NST cao → dùng L-BFGS.**
            👉 **Nếu muốn tùy chỉnh linh hoạt hoặc muốn thử nhiều biến thể → Adam là lựa chọn tiện hơn.**
            """)



In [ ]:
!pip install streamlit ngrok pyngrok

In [ ]:
!ngrok config add-authtoken 33XzMWuEzakeuxnB6brJBWUnXNB_3vqJtZVihToGSoY33VJZM

In [ ]:
# 1️⃣ Kill tunnel cũ
from pyngrok import ngrok
ngrok.kill()

# 2️⃣ Chạy Streamlit background (port 8502)
!nohup streamlit run app.py --server.port 8501 > streamlit.log 2>&1 &

# 3️⃣ Chờ Streamlit khởi động (5-10s)
import time
time.sleep(8)

# 4️⃣ Mở ngrok cho port 8501
public_url = ngrok.connect(8501)
print("Public URL:", public_url)